In [9]:
# Travel time cell
from datetime import datetime, timedelta
from pytz import timezone as ptz

DATE_FORMAT = "%Y-%m-%dT%H:%M:%-SZ"
oneDay = timedelta(1)

departureTimeOptions = {
    'now': lambda tz: datetime.now(tz).replace(minute=59, second=59).isoformat(),
    'morning': lambda tz: (datetime.now(tz).replace(hour=9, minute=0, second=0) + oneDay).isoformat(),
    'noon': lambda tz: (datetime.now(tz).replace(hour=12, minute=0, second=0) + oneDay).isoformat(),
    'afternoon': lambda tz: (datetime.now(tz).replace(hour=5, minute=0, second=0) + oneDay).isoformat(),
    'evening': lambda tz: (datetime.now(tz).replace(hour=7, minute=0, second=0) + oneDay).isoformat(),
}

print("now time: " + departureTimeOptions['now'](ptz('EST')))
print("morning time: " + departureTimeOptions['morning'](ptz('EST')))
print(datetime.now().timestamp())

now time: 2023-09-28T16:59:59.069458-05:00
morning time: 2023-09-29T09:00:00.069748-05:00
1695938076.069811


In [12]:
import requests
from const import GOOGLE_MAPS_API_KEY


COMPUTE_ROUTE_URL = 'https://routes.googleapis.com/directions/v2:computeRoutes'
MY_HOUSE = '3025 The Credit Woodlands, Mississauga, ON L5C 2V3'
SQUARE_ONE_ADDRESS = '100 City Centre Dr, Mississauga, ON L5B 2C9'
# travelTimes = {
#     now: lambda : datetime.datetime.now()
#     morning: lambda : datetime.datetime
# }


def computeRoutes(originAddress: str, destinationAddress: str, departureOption='now'):

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'DRIVE',
        'routingPreference': 'TRAFFIC_AWARE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
        'computeAlternativeRoutes': False,
        'routeModifiers': {
            'avoidTolls': True,
            'avoidHighways': False,
            'avoidFerries': True,
        },
        'languageCode': 'en-US',
        'units': 'IMPERIAL',
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return response.text


print(computeRoutes(MY_HOUSE, SQUARE_ONE_ADDRESS))

{
  "routes": [
    {
      "distanceMeters": 6988,
      "duration": "989s",
      "polyline": {
        "encodedPolyline": "omyhGtlceNkCuD?Gf@{@LCXN|EtFXd@DRCXq@tAb@l@Qf@wDdGyArCs@`As@r@eBhAw@X}@TkB^gAb@_Ar@q@p@k@|@aAtB}@bCk@tBo@rDS^UNe@Fa@?[RQZGRe@hEGxA?h@HhAHbEIbBKv@_@vAeC`GQlAEv@@zAEdAQjAY|@s@jAmEtGaArAiBjCa@RuBX]Nm@`Aa@r@yRiUyGwIaNmPo@{@eMcOaMeOi@i@{E{Fc@o@SEmGeHo@eA{EmGsDiEaGqGiLaNiFoGyCkDoRsU{LqOgIpOGRURGDgAD"
      }
    }
  ]
}



In [24]:


def computeTransitRoute(originAddress: str, destinationAddress: str, departureOption='now'):
    headers = {
        'content-type': 'application/json; application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.legs.steps.transitDetails',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'TRANSIT',
        'computeAlternativeRoutes': True,
        'transitPreferences': {
            'routingPreference': 'FEWER_TRANSFERS',
        },
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return response.text


print(computeTransitRoute(MY_HOUSE, SQUARE_ONE_ADDRESS))

{
  "routes": [
    {
      "legs": [
        {
          "steps": [
            {},
            {},
            {},
            {
              "transitDetails": {
                "stopDetails": {
                  "arrivalStop": {
                    "name": "Dundas St At Hurontario St",
                    "location": {
                      "latLng": {
                        "latitude": 43.5803729,
                        "longitude": -79.6160068
                      }
                    }
                  },
                  "arrivalTime": "2023-09-28T23:16:00Z",
                  "departureStop": {
                    "name": "Dundas St At The Credit Woodlands",
                    "location": {
                      "latLng": {
                        "latitude": 43.551466,
                        "longitude": -79.648757
                      }
                    }
                  },
                  "departureTime": "2023-09-28T23:05:00Z"
                },
           

In [18]:
import requests

headers = {
    'content-type': 'application/json; application/json',
    'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
    'X-Goog-FieldMask': 'routes.legs.steps.transitDetails',
}

data = '{\n  "origin": {\n    "address": "Humberto Delgado Airport, Portugal"\n  },\n  "destination": {\n    "address": "Basílica of Estrela, Praça da Estrela, 1200-667 Lisboa, Portugal"\n  },\n  "travelMode": "TRANSIT",\n  "computeAlternativeRoutes": true,\n  "transitPreferences": {\n     routingPreference: "LESS_WALKING",\n     allowedTravelModes: ["TRAIN"]\n  },\n}'.encode(
)

response = requests.post(
    'https://routes.googleapis.com/directions/v2:computeRoutes', headers=headers, data=data)

print(response.text)

{
  "routes": [
    {
      "legs": [
        {
          "steps": [
            {},
            {
              "transitDetails": {
                "stopDetails": {
                  "arrivalStop": {
                    "name": "Mq. Pombal - Av. Fontes P. Melo",
                    "location": {
                      "latLng": {
                        "latitude": 38.72625,
                        "longitude": -9.1497599999999988
                      }
                    }
                  },
                  "arrivalTime": "2023-09-28T23:00:18Z",
                  "departureStop": {
                    "name": "Aeroporto",
                    "location": {
                      "latLng": {
                        "latitude": 38.768961999999995,
                        "longitude": -9.1279779999999988
                      }
                    }
                  },
                  "departureTime": "2023-09-28T22:41:28Z"
                },
                "localizedValues": {
 